In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('c7').getOrCreate()

In [3]:
df = spark.read.format("csv")\
.option("header", "true")\
.option("inferSchema", "true")\
.load("retail-data/all/*.csv")\
.coalesce(5)

In [4]:
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: int, Country: string]

In [5]:
df.createOrReplaceTempView("dfTable")

In [6]:
df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [7]:
#simple aggregation
df.count() == 541909

True

In [8]:
# count
from pyspark.sql.functions import count

In [9]:
df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [10]:
# countDistinct
from pyspark.sql.functions import countDistinct

In [11]:
df.select(countDistinct("StockCode")).show(2)

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [12]:
# approx_count_distinct
from pyspark.sql.functions import approx_count_distinct

In [13]:
df.select(approx_count_distinct("StockCode",0.1)).show()

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3364|
+--------------------------------+



In [14]:
# first and last
from pyspark.sql.functions import first, last

In [15]:
df.select(first("StockCode"), last("StockCode")).show()

+-----------------------+----------------------+
|first(StockCode, false)|last(StockCode, false)|
+-----------------------+----------------------+
|                 85123A|                 22138|
+-----------------------+----------------------+



In [16]:
# min and max
from pyspark.sql.functions import min,max

In [17]:
df.select(min("Quantity"),max("Quantity")).show()

+-------------+-------------+
|min(Quantity)|max(Quantity)|
+-------------+-------------+
|       -80995|        80995|
+-------------+-------------+



In [18]:
#sum
from pyspark.sql.functions import sum

In [19]:
df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|      5176450|
+-------------+



In [20]:
#sumDistinct
from pyspark.sql.functions import sumDistinct

In [21]:
df.select(sumDistinct("Quantity")).show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|                 29310|
+----------------------+



In [22]:
#avg
from pyspark.sql.functions import sum, count,avg,expr

In [23]:
df.select(
    count("Quantity").alias("total_transactions"),
    sum("Quantity").alias("total_purchases"),
    avg("Quantity").alias("avg_purchases"),
    expr("mean(Quantity)").alias("mean_purchases")
).selectExpr(
    "total_purchases/total_transactions",
    "avg_purchases",
    "mean_purchases"
).show()

+--------------------------------------+----------------+----------------+
|(total_purchases / total_transactions)|   avg_purchases|  mean_purchases|
+--------------------------------------+----------------+----------------+
|                      9.55224954743324|9.55224954743324|9.55224954743324|
+--------------------------------------+----------------+----------------+



In [24]:
# variance and standard variation

In [25]:
from pyspark.sql.functions import var_pop,stddev_pop

In [26]:
from pyspark.sql.functions import var_samp, stddev_samp

In [27]:
df.select(var_pop("Quantity"), var_samp("Quantity"), stddev_pop("Quantity"), stddev_samp("Quantity")).show(2)

+------------------+------------------+--------------------+---------------------+
| var_pop(Quantity)|var_samp(Quantity)|stddev_pop(Quantity)|stddev_samp(Quantity)|
+------------------+------------------+--------------------+---------------------+
|47559.303646609354| 47559.39140929905|  218.08095663447864|   218.08115785023486|
+------------------+------------------+--------------------+---------------------+



In [28]:
#skewness and kurtosis
from pyspark.sql.functions import skewness,kurtosis

In [29]:
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+--------------------+------------------+
|  skewness(Quantity)|kurtosis(Quantity)|
+--------------------+------------------+
|-0.26407557610527843|119768.05495536518|
+--------------------+------------------+



In [30]:
#covariance and correlation

In [31]:
from pyspark.sql.functions import corr, covar_pop, covar_samp

In [32]:
df.select(
    corr("InvoiceNo","Quantity"), 
    covar_samp("InvoiceNo","Quantity"), 
    covar_pop("InvoiceNo","Quantity")
).show()

+-------------------------+-------------------------------+------------------------------+
|corr(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|
+-------------------------+-------------------------------+------------------------------+
|     4.912186085637639E-4|             1052.7280543913773|            1052.7260778752732|
+-------------------------+-------------------------------+------------------------------+



In [33]:
#aggregating complex types

In [34]:
from pyspark.sql.functions import collect_set,collect_list

In [35]:
df.agg(collect_set("Country"),collect_list("Country")).show()

+--------------------+---------------------+
|collect_set(Country)|collect_list(Country)|
+--------------------+---------------------+
|[Portugal, Italy,...| [United Kingdom, ...|
+--------------------+---------------------+



In [36]:
#grouping

In [37]:
df.groupBy("InvoiceNo","CustomerId").count().show()

+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536846|     14573|   76|
|   537026|     12395|   12|
|   537883|     14437|    5|
|   538068|     17978|   12|
|   538279|     14952|    7|
|   538800|     16458|   10|
|   538942|     17346|   12|
|  C539947|     13854|    1|
|   540096|     13253|   16|
|   540530|     14755|   27|
|   541225|     14099|   19|
|   541978|     13551|    4|
|   542093|     17677|   16|
|   543188|     12567|   63|
|   543590|     17377|   19|
|  C543757|     13115|    1|
|  C544318|     12989|    1|
|   544578|     12365|    1|
|   545165|     16339|   20|
|   545289|     14732|   30|
+---------+----------+-----+
only showing top 20 rows



In [38]:
#grouping with expressions

In [39]:
from pyspark.sql.functions import count

In [40]:
df.groupby("InvoiceNo").agg(
    count("Quantity").alias("quan"),
    expr("count(Quantity)")
).show()

+---------+----+---------------+
|InvoiceNo|quan|count(Quantity)|
+---------+----+---------------+
|   536596|   6|              6|
|   536938|  14|             14|
|   537252|   1|              1|
|   537691|  20|             20|
|   538041|   1|              1|
|   538184|  26|             26|
|   538517|  53|             53|
|   538879|  19|             19|
|   539275|   6|              6|
|   539630|  12|             12|
|   540499|  24|             24|
|   540540|  22|             22|
|  C540850|   1|              1|
|   540976|  48|             48|
|   541432|   4|              4|
|   541518| 101|            101|
|   541783|  35|             35|
|   542026|   9|              9|
|   542375|   6|              6|
|  C542604|   8|              8|
+---------+----+---------------+
only showing top 20 rows



In [41]:
#grouping with maps

In [42]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)")).show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

In [43]:
# testing date

In [44]:
from pyspark.sql.functions import col, to_date, date_format, to_utc_timestamp, unix_timestamp, from_unixtime

In [45]:
testDF = spark.createDataFrame([('12/1/2010 8:26',)], ['a'])

In [46]:
testDF.withColumn("date", to_date(from_unixtime(unix_timestamp(col('a'),'MM/d/yyyy HH:mm'), 'yyyy-MM-dd HH:mm'))).printSchema()

root
 |-- a: string (nullable = true)
 |-- date: date (nullable = true)



In [47]:
# window

In [48]:
from pyspark.sql.functions import col,to_date

In [49]:
dfWithDate = df.withColumn(
    "date", 
    to_date(
        from_unixtime(
            unix_timestamp(col('InvoiceDate'), 'MM/d/yyy HH:mm'), 'yyyy-MM-dd HH:mm')
    )
)

In [50]:
dfWithDate.createOrReplaceTempView("dfWithDate")

In [51]:
from pyspark.sql.window import Window

In [52]:
from pyspark.sql.functions import desc

In [53]:
windowSpec = Window.partitionBy("CustomerId", "date").orderBy(
    desc("Quantity")
).rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [54]:
from pyspark.sql.functions import max

In [55]:
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

In [56]:
from pyspark.sql.functions import dense_rank, rank

In [57]:
purchaseDenseRank = dense_rank().over(windowSpec)

In [58]:
purchaseRank = rank().over(windowSpec)

In [59]:
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
    col("CustomerId"),
    col("date"),
    col("Quantity"),
    purchaseRank.alias("quantityRank"),
    purchaseDenseRank.alias("quantityDenseRank"),
    maxPurchaseQuantity.alias("maxPurchaseQuantity")
).show()

+----------+----------+--------+------------+-----------------+-------------------+
|CustomerId|      date|Quantity|quantityRank|quantityDenseRank|maxPurchaseQuantity|
+----------+----------+--------+------------+-----------------+-------------------+
|     12346|2011-01-18|   74215|           1|                1|              74215|
|     12346|2011-01-18|  -74215|           2|                2|              74215|
|     12347|2010-12-07|      36|           1|                1|                 36|
|     12347|2010-12-07|      30|           2|                2|                 36|
|     12347|2010-12-07|      24|           3|                3|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|                 36|
|     12347|2010-12-07|      12|           4|                4|             

In [60]:
#grouping sets

In [61]:
dfNoNull = dfWithDate.drop()

In [62]:
dfNoNull.createOrReplaceTempView("dfNoNull")

In [64]:
spark.sql("""
SELECT CustomerId, StockCode, sum(Quantity)
FROM dfNoNull
GROUP BY CustomerId, StockCode
ORDER BY CustomerId DESC, StockCode DESC
""").show()

+----------+---------+-------------+
|CustomerId|StockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



In [65]:
spark.sql("""
SELECT CustomerId, StockCode, sum(Quantity)
FROM dfNoNull
GROUP BY CustomerId, StockCode GROUPING SETS((CustomerId,StockCode))
ORDER BY CustomerId DESC, StockCode DESC
""").show()

+----------+---------+-------------+
|CustomerId|StockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



In [66]:
spark.sql("""
SELECT CustomerId, StockCode, sum(Quantity)
FROM dfNoNull
GROUP BY CustomerId, StockCode GROUPING SETS((CustomerId,StockCode),())
ORDER BY CustomerId DESC, StockCode DESC
""").show()

+----------+---------+-------------+
|CustomerId|StockCode|sum(Quantity)|
+----------+---------+-------------+
|     18287|    85173|           48|
|     18287|   85040A|           48|
|     18287|   85039B|          120|
|     18287|   85039A|           96|
|     18287|    84920|            4|
|     18287|    84584|            6|
|     18287|   84507C|            6|
|     18287|   72351B|           24|
|     18287|   72351A|           24|
|     18287|   72349B|           60|
|     18287|    47422|           24|
|     18287|    47421|           48|
|     18287|    35967|           36|
|     18287|    23445|           20|
|     18287|    23378|           24|
|     18287|    23376|           48|
|     18287|    23310|           36|
|     18287|    23274|           12|
|     18287|    23272|           12|
|     18287|    23269|           36|
+----------+---------+-------------+
only showing top 20 rows



In [67]:
#rollups

In [68]:
rolledUpDF = dfNoNull.rollup("Date","Country").agg(sum("Quantity"))\
.selectExpr("Date","Country","`sum(Quantity)` as total_quantity")\
.orderBy("Date")

In [69]:
rolledUpDF.show()

+----------+--------------+--------------+
|      Date|       Country|total_quantity|
+----------+--------------+--------------+
|      null|          null|       5176450|
|2010-12-01|   Netherlands|            97|
|2010-12-01|          EIRE|           243|
|2010-12-01|        France|           449|
|2010-12-01|       Germany|           117|
|2010-12-01|     Australia|           107|
|2010-12-01|          null|         26814|
|2010-12-01|United Kingdom|         23949|
|2010-12-01|        Norway|          1852|
|2010-12-02|          EIRE|             4|
|2010-12-02|       Germany|           146|
|2010-12-02|          null|         21023|
|2010-12-02|United Kingdom|         20873|
|2010-12-03|        Poland|           140|
|2010-12-03|        France|           239|
|2010-12-03|       Germany|           170|
|2010-12-03|       Belgium|           528|
|2010-12-03|   Switzerland|           110|
|2010-12-03|          null|         14830|
|2010-12-03|         Spain|           400|
+----------

In [70]:
rolledUpDF.where("Country IS NULL").show()

+----------+-------+--------------+
|      Date|Country|total_quantity|
+----------+-------+--------------+
|      null|   null|       5176450|
|2010-12-01|   null|         26814|
|2010-12-02|   null|         21023|
|2010-12-03|   null|         14830|
|2010-12-05|   null|         16395|
|2010-12-06|   null|         21419|
|2010-12-07|   null|         24995|
|2010-12-08|   null|         22741|
|2010-12-09|   null|         18431|
|2010-12-10|   null|         20297|
|2010-12-12|   null|         10565|
|2010-12-13|   null|         17623|
|2010-12-14|   null|         20098|
|2010-12-15|   null|         18229|
|2010-12-16|   null|         29632|
|2010-12-17|   null|         16069|
|2010-12-19|   null|          3795|
|2010-12-20|   null|         14965|
|2010-12-21|   null|         15467|
|2010-12-22|   null|          3192|
+----------+-------+--------------+
only showing top 20 rows



In [71]:
rolledUpDF.where("Date IS NULL").show()

+----+-------+--------------+
|Date|Country|total_quantity|
+----+-------+--------------+
|null|   null|       5176450|
+----+-------+--------------+



In [72]:
# cube


In [73]:
from pyspark.sql.functions import sum

In [75]:
dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
.select("Date","Country","sum(Quantity)").orderBy("Date").show()

+----+--------------------+-------------+
|Date|             Country|sum(Quantity)|
+----+--------------------+-------------+
|null|            Portugal|        16180|
|null|               Japan|        25218|
|null|           Australia|        83653|
|null|             Germany|       117448|
|null|                 RSA|          352|
|null|           Hong Kong|         4769|
|null|              Cyprus|         6317|
|null|         Unspecified|         3300|
|null|           Singapore|         5234|
|null|                null|      5176450|
|null|               Spain|        26824|
|null|             Denmark|         8188|
|null|             Finland|        10666|
|null|     Channel Islands|         9479|
|null|             Lebanon|          386|
|null|  European Community|          497|
|null|                 USA|         1034|
|null|      Czech Republic|          592|
|null|United Arab Emirates|          982|
|null|              Norway|        19247|
+----+--------------------+-------

In [76]:
#grouping metadata

In [77]:
from pyspark.sql.functions import grouping_id, sum, expr

In [80]:
dfNoNull.cube("CustomerId","StockCode").agg(grouping_id(),sum(col("Quantity")))\
.orderBy(expr("grouping_id()").desc()).show()

+----------+---------+-------------+-------------+
|CustomerId|StockCode|grouping_id()|sum(Quantity)|
+----------+---------+-------------+-------------+
|      null|     null|            3|      5176450|
|      null|    22693|            2|        16172|
|      null|    90032|            2|           21|
|      null|    20749|            2|         1762|
|      null|    22376|            2|          255|
|      null|    22454|            2|          262|
|      null|    21657|            2|           30|
|      null|    21662|            2|           70|
|      null|    22791|            2|         7780|
|      null|    22065|            2|         6741|
|      null|   46000S|            2|         2092|
|      null|    21818|            2|         2610|
|      null|   82616C|            2|          321|
|      null|    84828|            2|         1079|
|      null|   90124B|            2|           12|
|      null|    21415|            2|          223|
|      null|    22474|         

In [81]:
#pivot

In [83]:
pivoted = dfWithDate.groupBy("date").pivot("Country").sum()

In [87]:
pivoted.where("date > '2011-12-05'").select("date","`USA_sum(CAST(`Quantity` AS BIGINT))`").show()

AnalysisException: 'syntax error in attribute name: `USA_sum(CAST(`Quantity` AS BIGINT))`;'

In [86]:
pivoted.columns

['date',
 'Australia_sum(CAST(`Quantity` AS BIGINT))',
 'Australia_sum(`UnitPrice`)',
 'Australia_sum(CAST(`CustomerID` AS BIGINT))',
 'Austria_sum(CAST(`Quantity` AS BIGINT))',
 'Austria_sum(`UnitPrice`)',
 'Austria_sum(CAST(`CustomerID` AS BIGINT))',
 'Bahrain_sum(CAST(`Quantity` AS BIGINT))',
 'Bahrain_sum(`UnitPrice`)',
 'Bahrain_sum(CAST(`CustomerID` AS BIGINT))',
 'Belgium_sum(CAST(`Quantity` AS BIGINT))',
 'Belgium_sum(`UnitPrice`)',
 'Belgium_sum(CAST(`CustomerID` AS BIGINT))',
 'Brazil_sum(CAST(`Quantity` AS BIGINT))',
 'Brazil_sum(`UnitPrice`)',
 'Brazil_sum(CAST(`CustomerID` AS BIGINT))',
 'Canada_sum(CAST(`Quantity` AS BIGINT))',
 'Canada_sum(`UnitPrice`)',
 'Canada_sum(CAST(`CustomerID` AS BIGINT))',
 'Channel Islands_sum(CAST(`Quantity` AS BIGINT))',
 'Channel Islands_sum(`UnitPrice`)',
 'Channel Islands_sum(CAST(`CustomerID` AS BIGINT))',
 'Cyprus_sum(CAST(`Quantity` AS BIGINT))',
 'Cyprus_sum(`UnitPrice`)',
 'Cyprus_sum(CAST(`CustomerID` AS BIGINT))',
 'Czech Republic_s